In [ ]:
pip install py7zr

In [ ]:
#Unzip zipped data set
import py7zr

with py7zr.SevenZipFile('/kaggle/input/cifar-10/train.7z', mode='r') as z:
    z.extractall(path='/kaggle/working')

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/cifar-10/trainLabels.csv')

In [ ]:
classes = {"airplane":1,"automobile":2,"bird":3,"cat":4,"deer":5,"dog":6,"frog":7,"horse":8,"ship":9,"truck":10}

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization, Activation, Conv2DTranspose
from keras.regularizers import l2
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import os
directory = '/kaggle/working/train'
train_images = np.zeros((50000,32,32,3))
train_labels = np.zeros((50000,1))
iter = 0
for filename in os.listdir(directory):
    img = image.load_img(os.path.join(directory,filename), target_size=(32, 32,3))
    x = image.img_to_array(img)
    name = filename.split('.')[0]
    train_images[iter] = x
    keys = df[df.id == int(name)].label.to_string().split(' ')[-1]
    train_labels[iter] = classes[keys]
    iter = iter + 1
    
train_images.shape
train_labels.shape

In [ ]:
train_images = train_images.reshape([-1, 32, 32, 3]).astype(np.float32) / 255
train_labels = train_labels.astype(np.int32)

In [ ]:
train_images=train_images.repeat(3,0)
train_labels=train_labels.repeat(3,0)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
plt.show()

In [ ]:
datagen = ImageDataGenerator( width_shift_range=0.1,height_shift_range=0.1, horizontal_flip=True,rotation_range=10, validation_split=0.05)

In [ ]:
train_labels_new = np.squeeze(train_labels)
train_labels_new = train_labels_new -1
train_labels_new = keras.utils.to_categorical(train_labels_new, 10)

In [ ]:
# The Model
mymodel = Sequential([
   Conv2D(32, 3, padding='same', input_shape = (32,32,3)),
    Activation('relu'),
    BatchNormalization(),
    Conv2D(64, 3, padding='same'),
    Activation('relu'),
    BatchNormalization(),
    Conv2D(64, 3),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
    Dropout(0.25),
    Conv2D(128, 5, padding='same'),
    Activation('relu'),
    BatchNormalization(),
    Conv2D(128, 5, padding = 'same'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
    Dropout(0.25),
    Conv2D(256, 5, padding='same', name = 'conv_last'),
    Activation('relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
    Dropout(0.25),
    Conv2D(512, 2),
    Flatten(),
    Dense(256, activation = 'relu',kernel_regularizer=l2(0.01)),
    Dropout(0.25),
    Dense(512, activation = 'relu',kernel_regularizer=l2(0.01)),
    Dropout(0.25),
    Dense(1024, activation = 'relu',kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dense(10, activation = 'softmax')
])
mymodel.compile(optimizer='adam',
              loss='kullback_leibler_divergence',#loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
history = mymodel.fit_generator(datagen.flow(train_images, train_labels_new,batch_size=128), epochs=10)

In [ ]:
rm -r /kaggle/working/train

In [ ]:
with py7zr.SevenZipFile('/kaggle/input/cifar-10/test.7z', mode='r') as z:
    z.extractall(path='/kaggle/working')

In [ ]:
class_dict = {0:"airplane",1:"automobile",2:"bird",3:"cat",4:"deer",5:"dog",6:"frog",7:"horse",8:"ship",9:"truck"}

In [ ]:
import csv
directory = '/kaggle/working/test'

with open('mySubmision.csv', 'w', newline='') as op:
    myWritter = csv.writer(op)
    myWritter.writerow(['id','label'])
    for filename in os.listdir(directory):
        img = image.load_img(os.path.join(directory,filename), target_size=(32, 32, 3))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = x/255.0
        id = filename.split('.')[0]
        prediction = np.argmax(mymodel.predict(x))
        myWritter.writerow([id,class_dict[prediction]])

    
print("Done")